In [1]:
from google.colab import drive
drive.mount('/content/a')

import pandas as pd
from time import time
import numpy as np
from IPython.display import Image

import keras
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input, LSTM, RepeatVector
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping

from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/a


Using TensorFlow backend.


In [0]:
input_file_path = 'a/My Drive/probData/LSTM/Prob/Prob_{}_{}.csv'
kmeans_out = 'a/My Drive/probData/LSTM/LSTM+Kmeans/{}_LSTM_Kmeans_{}_{}.csv'
dbscan_out = 'a/My Drive/probData/LSTM/LSTM+DBSCAN/{}_LSTM_DBSCAN_{}_{}.csv'

weekList = ['weekday_mor', 'weekday_aft', 'weekend_mor', 'weekend_aft']
monthList = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct']
cluster_num = [5]#[5]#[6, 7]
eps_List = [0.2, 0.3, 0.5, 0.7, 1, 0.02, 0.08, 0.0015, 0.01, 0.1]
min_samp = 7

porb_in_Path = 'a/My Drive/probData/M10/{}.csv'

In [0]:
def auto_LSTM(input_file_path, monthList, weekName):
    dfList = []
    for monthName in monthList:
        df_temp = pd.read_csv(input_file_path.format(monthName, weekName), index_col=0)
        dfList.append(df_temp)
        
    lstm_input = np.empty([2717, 10, 500])
    for ii in range(2717):
        for iii in range(10):
            lstm_input[ii, iii] = dfList[iii].iloc[ii, :].values
    lstm_input = lstm_input.transpose((0, 2, 1))
    
    # create model
    latent_dim = 10
    timesteps = 500
    input_dim = 10
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dim)(inputs)

    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    encoder.summary()
    sequence_autoencoder.summary()
    
    pretrain_epochs = 200
    batch_size = 256
    adam_opti = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    sequence_autoencoder.compile(optimizer=adam_opti, loss='mse')
    sequence_autoencoder.fit(lstm_input, lstm_input, batch_size=batch_size, epochs=pretrain_epochs, verbose = 2) #, callbacks=cb)
    sequence_autoencoder.save_weights('a/My Drive/AutoEncoder Clustering/lstm_weights_sy.h5')
    
    sequence_autoencoder.load_weights('a/My Drive/AutoEncoder Clustering/lstm_weights_sy.h5')
    y_pred = encoder.predict(lstm_input)
    
    return y_pred

def auto_kmeans(y_pred, cluster_core, label_output, prob_output, label, df_prob):
    kmeans = KMeans(n_clusters=cluster_core, random_state=0).fit(y_pred)
    result = pd.Series(kmeans.labels_)
    df_label = label.to_frame()
    df_label.loc[: ,"result"] = result
    df_label.to_csv(label_output)
    prob_dict = {}
#     r1 = pd.Series(kmeans.labels_).value_counts()   
#     r2 = pd.DataFrame(kmeans.cluster_centers_)  
#     r = pd.concat([r2, r1], axis = 1) 
#     df = df_prob.drop(['street_marker'],axis=1)
#     lkxs = metrics.silhouette_score(df, kmeans.labels_, metric='euclidean')

    for cluster_num in range(cluster_core):
        temp_list = np.zeros(shape=500)
        df_temp = df_label.loc[df_label.result==cluster_num]
        num_of_device = df_temp.shape[0]
        print(num_of_device)
        for marker in df_temp.street_marker.values.tolist():
            temp_device = df_prob.loc[df_prob.street_marker==marker].iloc[0, 1:].values
            temp_list = temp_list + temp_device
        prob_dict[str(cluster_num)] = temp_list/num_of_device

    df_cluster_prob = pd.DataFrame(prob_dict)
    df_cluster_prob.T.to_csv(prob_output) 
    
#     return lkxs
    return kmeans.inertia_
    
def auto_dbscan(y_pred, eps_val, min_samp, label_output, prob_output, label, df_prob):
    dbscan = DBSCAN(eps=eps_val, min_samples=min_samp).fit(y_pred)
    a = set(dbscan.labels_.tolist())
    
    result = pd.Series(dbscan.labels_)
    df_label = label.to_frame()
    df_label.loc[: ,"result"] = result
    df_label.to_csv(label_output)

    prob_dict = {}
    for cluster_num in a:
        temp_list = np.zeros(shape=500)
        df_temp = df_label.loc[df_label.result==cluster_num]
        num_of_device = df_temp.shape[0]
        print(num_of_device)
        for marker in df_temp.street_marker.values.tolist():
            temp_device = df_prob.loc[df_prob.street_marker==marker].iloc[0, 1:].values
            temp_list = temp_list + temp_device
        prob_dict[str(cluster_num)] = temp_list/num_of_device

    df_cluster_prob = pd.DataFrame(prob_dict)
    df_cluster_prob.T.to_csv(prob_output)

# **Manually DBSCAN**

In [0]:
weekName = weekList[3]
reference_path = porb_in_Path.format(weekName)

print("Start reading file: " + reference_path)
df_prob = pd.read_csv(reference_path)
label = df_prob.iloc[:, 0]
feature_input = df_prob.iloc[:, 1:]

y_pred = auto_LSTM(input_file_path, monthList, weekName)

Start reading file: a/My Drive/Research Data/TOP/probData/M10/weekend_aft.csv
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 500, 10)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 10)                840       
Total params: 840
Trainable params: 840
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 500, 10)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 10)                840       
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 500, 10)           0         


In [0]:
eps_val = 0.02
min_samp = 5

dbscan_label_output = dbscan_out.format('label', weekName, eps_val)
dbscan_prob_output = dbscan_out.format('prob', weekName, eps_val)
auto_dbscan(y_pred, eps_val, min_samp, dbscan_label_output, dbscan_prob_output, label, df_prob)

print("Finish processing file: " + reference_path)

2508
165
7
5
32
Finish processing file: a/My Drive/Research Data/TOP/probData/M10/weekend_aft.csv


# **Auto Script**

In [5]:
for weekName in weekList:
    reference_path = porb_in_Path.format(weekName)

    print("Start reading file: " + reference_path)
    df_prob = pd.read_csv(reference_path)
    label = df_prob.iloc[:, 0]
    feature_input = df_prob.iloc[:, 1:]
    print(df_prob)
    
    y_pred = auto_LSTM(input_file_path, monthList, weekName)
    
    lk = []

    # kmeans clustering
    for cluster_core in cluster_num:
        kmeans_label_output = kmeans_out.format('label', weekName, cluster_core)
        kmeans_prob_output = kmeans_out.format('prob', weekName, cluster_core)
        lkxs = auto_kmeans(y_pred, cluster_core, kmeans_label_output, kmeans_prob_output, label, df_prob)
        lk.append(lkxs)
    print('lk:', lk)
    
#     # DBSCAN clustering
#     for eps_val in eps_List:   
#         dbscan_label_output = dbscan_out.format('label', weekName, eps_val)
#         dbscan_prob_output = dbscan_out.format('prob', weekName, eps_val)
#         auto_dbscan(y_pred, eps_val, min_samp, dbscan_label_output, dbscan_prob_output, label, df_prob)

    print("Finish processing file: " + reference_path)  

Start reading file: a/My Drive/probData/M10/weekday_mor.csv
     street_marker    0    1    2  ...       496       497       498       499
0           13664E  1.0  1.0  1.0  ...  0.000000  0.000000  0.000000  0.000000
1             C798  1.0  1.0  1.0  ...  0.022059  0.022059  0.022059  0.022059
2           12483S  1.0  1.0  1.0  ...  0.005780  0.005780  0.005780  0.005780
3           13911W  1.0  1.0  1.0  ...  0.000000  0.000000  0.000000  0.000000
4            5050E  1.0  1.0  1.0  ...  0.003571  0.003571  0.003571  0.003571
5            9439S  1.0  1.0  1.0  ...  0.011111  0.011111  0.011111  0.011111
6            C7160  1.0  1.0  1.0  ...  0.000000  0.000000  0.000000  0.000000
7           14004E  1.0  1.0  1.0  ...  0.000000  0.000000  0.000000  0.000000
8           13304N  1.0  1.0  1.0  ...  0.002169  0.002169  0.002169  0.002169
9            5276E  1.0  1.0  1.0  ...  0.012658  0.012658  0.012658  0.012658
10          13251S  1.0  1.0  1.0  ...  0.000000  0.000000  0.000000  0

W0903 12:03:59.480671 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 12:03:59.512824 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 12:03:59.521044 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 12:03:59.952265 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500, 10)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
Total params: 840
Trainable params: 840
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500, 10)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 500, 10)           0         
____________________________________________

W0903 12:04:00.246969 140168659699584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0903 12:04:01.370355 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0903 12:04:01.455065 140168659699584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/200
 - 9s - loss: 0.0922
Epoch 2/200
 - 8s - loss: 0.0873
Epoch 3/200
 - 8s - loss: 0.0807
Epoch 4/200
 - 8s - loss: 0.0740
Epoch 5/200
 - 8s - loss: 0.0724
Epoch 6/200
 - 8s - loss: 0.0715
Epoch 7/200
 - 8s - loss: 0.0711
Epoch 8/200
 - 8s - loss: 0.0709
Epoch 9/200
 - 8s - loss: 0.0706
Epoch 10/200
 - 8s - loss: 0.0704
Epoch 11/200
 - 8s - loss: 0.0702
Epoch 12/200
 - 8s - loss: 0.0700
Epoch 13/200
 - 8s - loss: 0.0698
Epoch 14/200
 - 8s - loss: 0.0695
Epoch 15/200
 - 8s - loss: 0.0693
Epoch 16/200
 - 8s - loss: 0.0690
Epoch 17/200
 - 8s - loss: 0.0687
Epoch 18/200
 - 8s - loss: 0.0681
Epoch 19/200
 - 8s - loss: 0.0676
Epoch 20/200
 - 8s - loss: 0.0668
Epoch 21/200
 - 8s - loss: 0.0664
Epoch 22/200
 - 8s - loss: 0.0662
Epoch 23/200
 - 7s - loss: 0.0660
Epoch 24/200
 - 8s - loss: 0.0657
Epoch 25/200
 - 8s - loss: 0.0658
Epoch 26/200
 - 8s - loss: 0.0656
Epoch 27/200
 - 8s - loss: 0.0652
Epoch 28/200
 - 8s - loss: 0.0650
Epoch 29/200
 - 8s - loss: 0.0649
Epoch 30/200
 - 8s - lo